In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from local_functions import *
import math

In [ ]:
main_dir = Path(os.getcwd())
data_path = main_dir / "data"

In [ ]:
df_players = pd.read_csv(f'{data_path}/players.csv')
df_games = pd.read_csv(f'{data_path}/games.csv')
df_tackles = pd.read_csv(f'{data_path}/tackles.csv')
df_plays = pd.read_csv(f'{data_path}/plays.csv')
df_tracking = pd.read_csv(f'{data_path}/tracking_week_5.csv')

In [ ]:
# ex_gameId = df_games[(df_games['homeTeamAbbr']=='BAL') & (df_games['visitorTeamAbbr']=='CIN')]['gameId'].iloc[0]

# ex_playId = df_plays[(df_plays['gameId'] == ex_gameId) & 
#                      (df_plays['quarter'] == 1) &
#                      (df_plays['gameClock'] == '12:57')]['playId'].iloc[0]

# ex_game_play_id = str(ex_gameId) + '_' + str(ex_playId)

ex_game_play_id = '2022100901_117'

In [ ]:
fid_cols = ['gameId', 'playId', 'frameId']

# Standardize Tracking Data

In [ ]:
# Add player positions to tracking data
df_tracking = df_tracking.merge(df_players.loc[:, ['nflId', 'position']], on='nflId', how='left')

# Create game/play ID for easier filtering
df_tracking['game_play_id'] = df_tracking['gameId'].astype(str) + "_" + df_tracking['playId'].astype(str)

# Add indicator column for the ball carrier in the tracking data
df_tracking = df_tracking.merge(df_plays.loc[:, ['gameId', 'playId', 'ballCarrierId']], left_on=['gameId','playId','nflId'], right_on=['gameId','playId','ballCarrierId'], how='left')
df_tracking['is_ballcarrier'] = df_tracking['nflId'] == df_tracking['ballCarrierId']

In [ ]:
# Standardize x/y coordinates - all plays going to the right
df_tracking['x_std'] = np.where(df_tracking['playDirection'] == 'left', 120 - df_tracking['x'], df_tracking['x'])
df_tracking['y_std'] = np.where(df_tracking['playDirection'] == 'left', 160/3 - df_tracking['y'], df_tracking['y'])

In [ ]:
# Standardize player orientation - all plays going to the right
df_tracking['o_std'] = np.where(df_tracking['playDirection'] == 'left', df_tracking['o'] + 180, df_tracking['o'])
df_tracking['o_std'] = np.where(df_tracking['o_std'] > 360, df_tracking['o_std'] - 360, df_tracking['o_std'])
df_tracking['o_rad'] = np.radians(df_tracking['o_std'])
df_tracking['o_x'] = np.sin(df_tracking['o_rad'])
df_tracking['o_y'] = np.cos(df_tracking['o_rad'])

In [ ]:
# Standardize player movement direction - all plays going to the right
df_tracking['dir_std'] = np.where(df_tracking['playDirection'] == 'left', df_tracking['dir'] + 180, df_tracking['dir'])
df_tracking['dir_std'] = np.where(df_tracking['dir_std'] > 360, df_tracking['dir_std'] - 360, df_tracking['dir_std'])
df_tracking['dir_rad'] = np.radians(df_tracking['dir_std'])
df_tracking['dir_x'] = np.sin(df_tracking['dir_rad'])
df_tracking['dir_y'] = np.cos(df_tracking['dir_rad'])

In [ ]:
# Create columns for speed and acceleration in x/y direction

df_tracking['v_x'] = df_tracking['dir_x'] * df_tracking['s']
df_tracking['v_y'] = df_tracking['dir_y'] * df_tracking['s']

df_tracking['a_x'] = df_tracking['dir_x'] * df_tracking['a']
df_tracking['a_y'] = df_tracking['dir_y'] * df_tracking['a']

df_tracking['v_theta'] = np.arctan(df_tracking['v_y'] / df_tracking['v_x'])
df_tracking['v_theta'] = np.where(df_tracking['v_theta'].isnull(), 0, df_tracking['v_theta'])

In [ ]:
# Join ball carrier tracking to total tracking - allows for calculating distance, speed, etc of player X relative to the ball carrier
df_bc_frames = df_tracking[df_tracking['is_ballcarrier']].drop_duplicates(['gameId','playId','frameId','nflId']).loc[:,['gameId', 'playId', 'frameId', 'nflId', 'x_std', 'y_std', 'dir_std', 'o_std', 's', 'a']].reset_index(drop=True)
df_tracking = df_tracking.merge(df_bc_frames.drop('nflId', axis=1), on = ['gameId','playId','frameId'], how = 'inner', suffixes = ('', '_bc'))

# Field Control

In [ ]:
# Calculate player influence radius
s_max = 13
delta_t = 0.5
min_radius = 4
max_radius = 10
radius_range = max_radius - min_radius
max_dist_from_ball = 20

df_tracking['dist_to_bc'] = euclidean_distance(df_tracking['x_std'], df_tracking['y_std'], df_tracking['x_std_bc'], df_tracking['y_std_bc'])
df_tracking['s_ratio'] = df_tracking['s'] / s_max
df_tracking['x_next'] = df_tracking['x_std'] + df_tracking['v_x'] * delta_t
df_tracking['y_next'] = df_tracking['y_std'] + df_tracking['v_y'] * delta_t
df_tracking['radius_of_influence'] = min_radius + np.power(df_tracking['dist_to_bc'],3) * radius_range / max_dist_from_ball
df_tracking['radius_of_influence'] = np.where(df_tracking['radius_of_influence'] > max_radius, max_radius, df_tracking['radius_of_influence'])

In [ ]:
#df_tracking.loc[:, ['frameId', 'nflId', 'x_std', 'y_std', 'x_std_bc', 'y_std_bc', 's', 'v_theta', 'v_x', 'v_y', 's_ratio', 'x_next', 'y_next', 'radius_of_influence']]

In [ ]:
df_field_grid = create_field_grid()

In [ ]:
df_example_tracking = df_tracking[df_tracking['game_play_id'] == ex_game_play_id].reset_index(drop=True)
df_example_for_control = df_example_tracking[df_example_tracking['nflId'].notnull()].loc[:,['gameId', 'playId', 'frameId', 'nflId', 'club', 'x_next', 'y_next', 'v_theta', 'radius_of_influence', 's_ratio']].reset_index(drop=True)

In [ ]:
player_frame_influences = []
#df_control = pd.DataFrame()

for index, row in df_example_for_control.iterrows():
    #print(index)
    
    row_influence = compute_player_zoi(row, df_field_grid)
    player_frame_influences.append(row_influence)

    #df_influence = compute_player_zoi(row)
    #df_control = pd.concat([df_control, df_influence])

df_example_for_control['influence'] = player_frame_influences

In [ ]:
df_example_for_control = df_example_for_control.merge(df_games.loc[:,['gameId', 'homeTeamAbbr']], on = 'gameId')


In [ ]:
df_control_explode = df_example_for_control.explode('influence')
df_control_explode['influence'] = df_control_explode['influence'].astype('float')

In [ ]:
df_control_explode.influence.hist()

In [ ]:
c_field_grid = pd.DataFrame()

for i in range(len(df_example_for_control.index)):
    c_field_grid = pd.concat([c_field_grid, df_field_grid])


In [ ]:
df_control_all = pd.concat([df_control_explode.reset_index(drop=True), c_field_grid.reset_index(drop=True)], axis=1)

In [ ]:
df_bc_frames

In [ ]:
bc_radius = 3

df_bc_frames['bc_xr_min'] = df_bc_frames['x_std'] - bc_radius
df_bc_frames['bc_xr_max'] = df_bc_frames['x_std'] + bc_radius

df_bc_frames['bc_yr_min'] = df_bc_frames['y_std'] - bc_radius
df_bc_frames['bc_yr_max'] = df_bc_frames['y_std'] + bc_radius

bc_radius_cols = ['bc_xr_min', 'bc_xr_max', 'bc_yr_min', 'bc_yr_max']

In [ ]:
df_control_all = df_control_all.merge(df_bc_frames.loc[:,fid_cols + bc_radius_cols], on = fid_cols)

In [ ]:
df_control_bc_radius = df_control_all[(df_control_all['x'] >= df_control_all['bc_xr_min']) &
               (df_control_all['x'] <= df_control_all['bc_xr_max']) &
               (df_control_all['y'] >= df_control_all['bc_yr_min']) &
               (df_control_all['y'] <= df_control_all['bc_yr_max'])]

In [ ]:
df_control_bc_inf_agg = df_control_bc_radius.groupby(fid_cols + ['nflId']).agg({'influence' : 'mean'}).reset_index()

In [ ]:
scale_column_to_100(dataframe = df_control_bc_inf_agg, column_name = 'influence')

In [ ]:
df_control_all['influence'] = np.where(df_control_all['club'] == df_control_all['homeTeamAbbr'], -1 * df_control_all['influence'], df_control_all['influence'])
df_control_agg = df_control_all.groupby(['gameId', 'playId', 'frameId', 'club', 'x', 'y']).agg({'influence' : 'sum'}).reset_index().rename(columns={'influence' : 'control'})
df_control_agg['control'] = 1 / (1 + np.exp(df_control_agg['control']))

In [ ]:
df_control_1frame = df_control_agg[(df_control_agg['frameId']==75)]# & (df_control_agg['club']=='PIT')]

from plotnine import ggplot, geom_raster, scale_fill_gradient2, aes

# Assuming df_colors and game_ are already defined in your Python script

away_team = 'BUF'
away_color = 'blue'

home_team = 'PIT'
home_color = 'black'


play_frames = (
    ggplot()
    + geom_raster(
        data=df_control_1frame,
        mapping=aes(x='x', y='y', fill='control'),
        alpha=0.7,
        interpolate=True
    )
    + scale_fill_gradient2(
        #low=df_colors['away_1'],
        #high=df_colors['home_1'],
        low= away_color,
        high= home_color,
        mid="white",
        midpoint=0.5,
        name="Team Field Control",
        limits=[0, 1],
        breaks=[0, 1],
        #labels=[game_['visitorTeamAbbr'], game_['homeTeamAbbr']]
        labels=[away_team, home_team]
    )
)

# You can then show the plot using play_frames
print(play_frames)


# Angle Diffs

In [ ]:
# Calculate the orientation of each player relative to the ball carrier
df_tracking = calc_angle_diff(input_df = df_tracking, 
                              xc = 'x_std', 
                              yc = 'y_std', 
                              anglec = 'o', 
                              xc_ref = 'x_std_bc', 
                              yc_ref = 'y_std_bc', 
                              new_name_suffix = 'bc')

In [ ]:
# Calculate the movement direction of each player relative to the ball carrier
df_tracking = calc_angle_diff(input_df = df_tracking, 
                              xc = 'x_std', 
                              yc = 'y_std', 
                              anglec = 'dir', 
                              xc_ref = 'x_std_bc', 
                              yc_ref = 'y_std_bc', 
                              new_name_suffix = 'bc')

In [ ]:
# Calculate the point where a blocker projects onto a defender's path to the ball carrier

target = (1,6)
defender = (3,10)
blocker = (4,7)

blocker_projection = perpendicular_projection(blocker, line_equation(defender, target))

In [ ]:
x, y = 37, 16
xd = 7
yd = 3
facing_angle = 200  # in degrees
maxX = 120
maxY = 53.3
minX = 0
minY = 0


# Calculate potential blocker region for a defender based on their orientation angle
triangle_points = project_triangle(x, y, facing_angle, xd, yd, maxX, maxY, minX, minY)

print(triangle_points)

In [ ]:
# Determine if an offensive player is a potential blocker
point_to_check = (32, 15)

is_point_in_triangle(point_to_check, *triangle_points)

In [ ]:
[(37,16)]+triangle_points

In [ ]:
# Plotting
triangle_points_plot = [(37,16)]+triangle_points

points_x, points_y = zip(*triangle_points_plot + [triangle_points_plot[0]])  # Close the triangle

plt.plot(points_x, points_y, marker='o', linestyle='-', color='b', label='Projected Triangle')
plt.scatter(32,15)

# Play Animation

In [ ]:
ex_game_play_id = '2022100913_172'
#ex_game_play_id = '2022100913_55'

In [ ]:
animate_tracking_data(tracking_df = df_tracking, 
                      id_game_play = ex_game_play_id,
                      x_col = 'x_std',
                      y_col = 'y_std',
                      dir_col = 'dir_std',
                      dir_arrow_metric = 's',
                      o_col = 'o_std')